# Data Cleaning

## Libraries

In [19]:
# Libraries
import pandas as pd
import numpy as np
import re
from datetime import datetime as dt

## Original Dataset

In [3]:
netflix = pd.read_csv("../data/raw-data/Netflix_Titles.csv")
netflix.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


### Identifying NaNs & Column Types

Once we have loaded the initial DF, first of all we should check that the column types on the DF are OK. After that, we will identify if there are NaN values on the DF and decide what we will do with the missing values

In [4]:
netflix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6234 entries, 0 to 6233
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       6234 non-null   int64 
 1   type          6234 non-null   object
 2   title         6234 non-null   object
 3   director      4265 non-null   object
 4   cast          5664 non-null   object
 5   country       5758 non-null   object
 6   date_added    6223 non-null   object
 7   release_year  6234 non-null   int64 
 8   rating        6224 non-null   object
 9   duration      6234 non-null   object
 10  listed_in     6234 non-null   object
 11  description   6234 non-null   object
dtypes: int64(2), object(10)
memory usage: 584.6+ KB


In [5]:
netflix.isna().sum()

show_id            0
type               0
title              0
director        1969
cast             570
country          476
date_added        11
release_year       0
rating            10
duration           0
listed_in          0
description        0
dtype: int64

Although there several NaN values, the ones that have a higher impact in our research are from the column **country**. Let's investigate a bit more and see which impact might have if we drop those values.

In [6]:
print(f"Ratio Impact on Country column: {round(netflix.country.isna().sum() / netflix.country.notna().sum() * 100, 2)}")

Ratio Impact on Country column: 8.27


The impact is not that much, but if we drop those values we could impact negatively on the **type** column, as we might drop a considerable amount of data for the _Films_ or _TV Shows_

In [7]:
def impact_type_column(type):
    """
    Input: The column type
    Output: Returns the impact ratio if we drop country NaN values on the type column
    """
    
    
    shows_no_country = netflix[netflix.country.isna()].type.value_counts()[0]
    films_no_country = netflix[netflix.country.isna()].type.value_counts()[1]
    
    total_shows = netflix.type.value_counts()[1]
    total_films = netflix.type.value_counts()[0]
    
    ratio_shows = round(shows_no_country / total_shows * 100, 2)
    ratio_films = round(films_no_country / total_films * 100, 2)
    
    solution = print(f"Shows Impacted: {ratio_shows}\tFilms Impacted: {ratio_films}")
    
    return solution

In [8]:
impact_type_column(type)

Shows Impacted: 14.27	Films Impacted: 4.57


The impact it will have on the _Shows_ it will be almost the 15% of the content. For that reason, we will keep NaN values from **countries** column.

### Drop Columns

Some columns won't be necessary for our work, for that reason we will drop them. The columns droped are the following ones:
* **show_id**: Id number of the content.
* **description**: Description of the show.

In [9]:
netflix = netflix.drop(columns=["show_id", "description"])
netflix.head()

,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in
0,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies"
1,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy
2,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV
3,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV
4,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies


### Change DateFormat column

Now, the **date_added** doesn't allow us to properly filter it, as it shows the full date. We would need something less specific, like the **release_year** column. For that, we will apply those changes:
1. Create **year_added** column: Allowing us to have only the year information.
2. Modify **date_added** column: Showing only the month and year when it was added.

In [10]:
netflix["date_added"] = pd.to_datetime(netflix["date_added"])
netflix["year_added"] = netflix["date_added"].dt.strftime("%Y")
netflix["date_added"] = netflix["date_added"].dt.strftime("%Y-%m")
netflix.head()

,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,year_added
0,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",2019-09,2019,TV-PG,90 min,"Children & Family Movies, Comedies",2019
1,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,2016-09,2016,TV-MA,94 min,Stand-Up Comedy,2016
2,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,2018-09,2013,TV-Y7-FV,1 Season,Kids' TV,2018
3,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,2018-09,2016,TV-Y7,1 Season,Kids' TV,2018
4,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,2017-09,2017,TV-14,99 min,Comedies,2017


## Mergin DataFrames

Although our initial DF contains a lot of information, let's add some more. We have three diferent .csv files that might be added to this DF. Let's add them one by one.

### IMDb.csv

As Netflix doesn't have ratings per se, we would need to get this information from external sources, in this case from **IMDB**. We will add the rating and the amount of votes for each Film/TV Show


_The process for obtaining this information is done on the `2. web-scraping-IMDb.ipynb` notebook_

In [11]:
# Importing IMDB DF
IMDB = pd.read_csv("../data/raw-data/IMDb_results_nov-22-2020.csv", index_col=0)

# Joinig both tables
netflix = netflix.join(IMDB)

# Cleaning the DF mantaining the desired columns
netflix = netflix.drop(columns=["IMDB_titleID", "IMDB_title_name"])
netflix = netflix.rename(columns={"IMDB_rating":"IMDb_rating"})
netflix.head()

,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,year_added,IMDb_rating
0,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",2019-09,2019,TV-PG,90 min,"Children & Family Movies, Comedies",2019,3.2 based on 294 user ratings
1,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,2016-09,2016,TV-MA,94 min,Stand-Up Comedy,2016,5.0 based on 21 user ratings
2,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,2018-09,2013,TV-Y7-FV,1 Season,Kids' TV,2018,"7.9 based on 5,331 user ratings"
3,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,2018-09,2016,TV-Y7,1 Season,Kids' TV,2018,6.0 based on 830 user ratings
4,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,2017-09,2017,TV-14,99 min,Comedies,2017,"5.2 based on 5,496 user ratings"


Now that this new column is added, we see another problem: the rating and the votes are on the same string. From this column, we need to extract this information and create two new columns, one for **rating** and another for **votes**.

In [29]:
# First isolation of the rating (stars) of the column IMDb_rating
# Create an empty list for storing the rates
rating = []

# Loop that iterates through the column and extracts the values
# Some values are NaN, that's why we need to add an try/except clause
for rate in netflix["IMDb_rating"]:
    try:
        rating.append(float(re.findall(r"\d{1}\.\d{1}", rate)[0]))
    except:
        rating.append(np.NAN)
        
netflix["IMDb_rate"] = rating

# Second isolation of the rating (votes) of the column IMDb_rating
# Values above 1K have a comma as a separator. Let's remove that comma

netflix["IMDb_rating"] = netflix["IMDb_rating"].str.replace(",", "")

# Create an empty list for storing the votes
votes = []

for rate in netflix["IMDb_rating"]:
    try:
        votes.append(re.findall(r"\d+", rate))
    except:
        votes.append(np.nan)
        

# The list votes contains both the rating and the amount of votes
# If there are ratings, it means that the votes will be too
# In that case, we will find the number of votes on the 2nd index
# We will append that index into a new list
correct_votes = []
for v in votes:
    try:
        correct_votes.append(int(v[2]))
    except:
        correct_votes.append(np.nan)
        
netflix["IMDb_votes"] = correct_votes

# Convert the float of ["IMDb_votes"] to int with '.astype' from Pandas
netflix["IMDb_votes"] = netflix["IMDb_votes"].astype("Int64")

# Drop initial column to avoid confusion
netflix = netflix.drop(columns="IMDb_rating")
netflix.head()

,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,year_added,IMDb_rate,IMDb_votes
0,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",2019-09,2019,TV-PG,90 min,"Children & Family Movies, Comedies",2019,3.2,294
1,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,2016-09,2016,TV-MA,94 min,Stand-Up Comedy,2016,5.0,21
2,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,2018-09,2013,TV-Y7-FV,1 Season,Kids' TV,2018,7.9,5331
3,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,2018-09,2016,TV-Y7,1 Season,Kids' TV,2018,6.0,830
4,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,2017-09,2017,TV-14,99 min,Comedies,2017,5.2,5496


### StreamingPlatforms_Movies/Shows.csv

In [6]:
# Importing DF that indicates if a TV Show / Film appears in different streaming platforms

# Streaming Movies DF
streaming_platforms_movies = pd.read_csv("../data/raw-data/StreamingPlatforms_Movies.csv", index_col=0)
streaming_platforms_movies = streaming_platforms_movies.drop(columns=["Language", "Runtime", "Directors", "Genres", "Type", "IMDb", "Rotten Tomatoes", "ID", "Country", "Age"])

# Series Movies DF
streaming_platforms_series = pd.read_csv("../data/raw-data/StreamingPlatforms_TV_Shows.csv", index_col=0)
streaming_platforms_series = streaming_platforms_series.drop(columns=["IMDb", "Rotten Tomatoes", "type", "Age"])

# Merging both DF
streamings_complete = pd.concat([streaming_platforms_movies, streaming_platforms_series])
streamings_complete = streamings_complete.reset_index(drop=True)
streamings_complete = streamings_complete.rename(columns={"Title":"title", "Year":"year", "Age":"age"})

# Merge previous DF with the main (netflix)
netflix = netflix.merge(streamings_complete, on="title", how="left")

In [7]:
# Filling the NA values for being able to append from the other row if value is missing
netflix["year"] = netflix["year"].fillna(0).astype(int)

def correct_release_year(row):
    
    """
    Input: Row of the DF
    Output: If the first condition is met, the value will be appended; if it's missing will append the other row value
    
    """
    if row["year"] > 1:
        return row["year"]
    if row["year"] == 0:
        return row["release_year"]

In [8]:
# Creating new column with the function created above
netflix["correct_release_year"] = netflix.apply(correct_release_year, axis=1)

# Drop old year columns to avoid confusion
netflix = netflix.drop(columns=["release_year", "year"])

In [9]:
# Converting Floats from the Streaming Platforms to Int
# There are NaN values, that's why I use '.astype' from Pandas
netflix[["Netflix", "Hulu", "Prime Video", "Disney+"]] = netflix[["Netflix", "Hulu", "Prime Video", "Disney+"]].astype("Int64")

### Data Storage

In [12]:
# Export the file to CSV for better manipulation
netflix.to_csv("../data/netflix_cleandata.csv", index=False)